In [1]:
### 🚀 RUN ME BEFORE YOU START WORKING ON THIS NOTEBOOK ⚠️

import helpers
helpers.initialize()

### 🚀 RUN ME BEFORE YOU START WORKING ON THIS NOTEBOOK ⚠️

🔄 Initializing Course environment...
🔁 Autoreload enabled: modules will reload automatically when changed
🔕 Suppressed future deprecation warnings
📝 Logging configured
📊 Pandas display settings configured for better output
🔍 Looking for .env file at: /workspaces/llmops-course/.env
✅ Successfully loaded environment variables from /workspaces/llmops-course/.env
📋 Loaded variables: GEMINI_API_KEY=****sbqo, DATABRICKS_HOST=https..., DATABRICKS_TOKEN=****e2-2
⚙️ Disabled MLflow system metrics logging
📔 Disabled MLflow notebook display (avoids VSCode bugs)

🎉 All systems go! Your Course environment is ready for learning!


In [3]:
import mlflow
import os

mlflow.set_tracking_uri("databricks")
print(mlflow.search_experiments())
mlflow.set_experiment("/Users/zoltanat@nordquant.com/Summarizer Extension")


[<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/239898367922644', creation_time=1741256411137, experiment_id='239898367922644', last_update_time=1741266426250, lifecycle_stage='active', name='/Users/zoltanat@nordquant.com/Summarizer Extension', tags={'mlflow.experiment.sourceName': '/Users/zoltanat@nordquant.com/Summarizer '
                                 'Extension',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'zoltanat@nordquant.com',
 'mlflow.ownerId': '8919370611258914'}>, <Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/239898367922645', creation_time=1741256444861, experiment_id='239898367922645', last_update_time=1741256444861, lifecycle_stage='active', name='/Users/zoltanat@nordquant.com/Summarizer Extension 2', tags={'mlflow.experiment.sourceName': '/Users/zoltanat@nordquant.com/Summarizer '
                                 'Extension 2',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'zoltanat@

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/239898367922644', creation_time=1741256411137, experiment_id='239898367922644', last_update_time=1741266426250, lifecycle_stage='active', name='/Users/zoltanat@nordquant.com/Summarizer Extension', tags={'mlflow.experiment.sourceName': '/Users/zoltanat@nordquant.com/Summarizer '
                                 'Extension',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'zoltanat@nordquant.com',
 'mlflow.ownerId': '8919370611258914'}>

In [4]:

import glob
import pandas as pd
from pathlib import Path

# Recursively find all HTML files in assets directory
html_files = glob.glob("assets/articles_short/*.html")

# Create list to store file data
file_data = []

# Read each HTML file
for file_path in html_files:
    content = open(file_path, 'r', encoding='utf-8').read()
    file_data.append({
        'file_name': Path(file_path).name,
        'text': content
    })

# Create DataFrame
eval_df = pd.DataFrame(file_data)


path = 'assets/articles_full_length/ai_relationships.html'
text = open(path).read()

input_example = pd.DataFrame({'text': [text]})


In [5]:
from cloudsummarizer import SerializableArticleSummarizerModel

prompt = "Summarize the article in 2 sentences: {text}"
max_output_tokens = 200
simple_model = SerializableArticleSummarizerModel(prompt=prompt, max_output_tokens=max_output_tokens)
eval_df["summaries"] = simple_model.predict(eval_df)

eval_df

file_name  \
0  the_end_of_internet_search.html   
1         electricity_demants.html   
2            ai_relationships.html   
3            prompt_injection.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [6]:
from mlflow.metrics import MetricValue, make_metric
from mlflow.metrics.base import standard_aggregations
from cloudsummarizer import REFUSAL_TEXT

def model_refused_to_respond(predictions, targets):
    scores = []
    for prediction in predictions:
        if prediction == REFUSAL_TEXT:
            scores.append(1)
        else:
            scores.append(0)

    return MetricValue(
        scores=scores,
        aggregate_results=standard_aggregations(scores),
    )


# Create an EvaluationMetric object.
refusal_rate_metric = make_metric(
    eval_fn=model_refused_to_respond, greater_is_better=False, name="refusal_rate"
)

path = Path('assets/articles_full_length/ai_relationships.html')
text = path.read_text()

input_example = pd.DataFrame({'text': [text]})


In [7]:
with mlflow.start_run(run_name="Simple Model") as run:
    eval_result = mlflow.evaluate(
        data=eval_df,
        predictions="summaries",
        targets="text",
        model_type="text-summarization",
        extra_metrics=[
            refusal_rate_metric
        ],
    )

    mlflow.log_param("max_output_tokens", max_output_tokens)
    mlflow.log_param("prompt", prompt)

    mlflow.pyfunc.log_model(
        "article_summarizer",
        python_model=simple_model,
        input_example=input_example,
        signature=mlflow.models.infer_signature(
            input_example,
            simple_model.predict(input_example)
        ),
        infer_code_paths=True,
        registered_model_name="article_summarizer",
    )


2025/03/06 13:10:46 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
PyTorch version 2.6.0 available.
Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint
Device set to use cpu
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
2025/03/06 13:11:00 INFO mlflow.pyfunc: Validating input example against model signature
2025/03/06 13:11:10 INFO mlflow.models.model: Found the following environment variables used during model inference: [GEMINI_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Registered model 'article_summarizer' already exists. Creating a new version of this model...
2025/03/06 13:11:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: article_summarizer, version 3
Created version '3' of model 'article_summarizer'.


🏃 View run Simple Model at: https://adb-2424489540300441.1.azuredatabricks.net/ml/experiments/239898367922644/runs/2a5ec99406d246e9a4de2a2f12fb5e39
🧪 View experiment at: https://adb-2424489540300441.1.azuredatabricks.net/ml/experiments/239898367922644


In [8]:
from cloudsummarizer import secure_prompt
max_output_tokens = 200
secure_model = SerializableArticleSummarizerModel(prompt=secure_prompt, max_output_tokens=max_output_tokens)
eval_df["summaries"] = secure_model.predict(eval_df)


In [9]:
with mlflow.start_run(run_name="Secure Model") as run:
    eval_result = mlflow.evaluate(
        data=eval_df,
        predictions="summaries",
        targets="text",
        model_type="text-summarization",
        extra_metrics=[
            refusal_rate_metric
        ],
    )

    mlflow.log_param("max_output_tokens", max_output_tokens)
    mlflow.log_param("prompt", prompt)

    model_info = mlflow.pyfunc.log_model(
        "article_summarizer",
        python_model=secure_model,
        input_example=input_example,
        signature=mlflow.models.infer_signature(
            input_example,
            secure_model.predict(input_example)
        ),
        infer_code_paths=True,
        registered_model_name="article_summarizer",
    )


2025/03/06 13:11:19 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
Using default tokenizer.
2025/03/06 13:11:24 INFO mlflow.pyfunc: Validating input example against model signature
2025/03/06 13:11:34 INFO mlflow.models.model: Found the following environment variables used during model inference: [GEMINI_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Registered model 'article_summarizer' already exists. Creating a new version of this model...
2025/03/06 13:11:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: article_summarizer, version 4
Created version '4' of model 'article_summarizer'.


🏃 View run Secure Model at: https://adb-2424489540300441.1.azuredatabricks.net/ml/experiments/239898367922644/runs/a3f3d00185d54e0782f3bf99d606f540
🧪 View experiment at: https://adb-2424489540300441.1.azuredatabricks.net/ml/experiments/239898367922644


In [10]:
# Remove production stage tag from all versions
client = mlflow.tracking.MlflowClient()
for mv in client.search_model_versions(f"name='article_summarizer'"):
    try:
        client.delete_model_version_tag(mv.name, mv.version, "stage")
    except:
        pass  # Tag may not exist

# Add production stage tag to current version
client.set_model_version_tag(
    name="article_summarizer",
    version=model_info.registered_model_version,
    key="stage",
    value="production"
)
model_info.registered_model_version

'4'

In [11]:
# Get the latest production model
client = mlflow.tracking.MlflowClient()
model_versions = client.search_model_versions("name='article_summarizer'")
sorted_versions = sorted(model_versions, key=lambda x: int(x.version), reverse=True)

production_model = None
for version in sorted_versions:
    if "stage" in version.tags and version.tags["stage"] == "production":
        print(f'Production model found: {version.version}')
        print(f'Production Model URI:\n========\nmodels:/article_summarizer/{version.version}\n========')
        break
else:
    raise Exception("No production model found")


Production model found: 4
Production Model URI:
models:/article_summarizer/4
